In [1]:
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from sqlalchemy import create_engine

In [2]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


c:\python27\arcgis10.2\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
engine = create_engine("mssql+pyodbc://gt_QAQC_Tools")

In [4]:
sql = "Select CertNum, SampleID, SampleType, Reference, DateFinalised, Pass2mm, Pass75um, [WT. SAMPLE] from \
(Select a.CertNum, a.DateFinalised, b.SampleID, b.SampleType, b.Reference, b.Element, b.Value \
from dbo.LabJobCert a join dbo.LabJobResults b on a.CertNum = b.CertNum \
where DATEPART(YYYY, a.DateFinalised) = 2017 and b.Element in ('Pass2mm','Pass75um', 'WT. Sample')) as sourceTable PIVOT \
(MIN(Value) for Element in ([Pass2mm], [Pass75um], [WT. SAMPLE])) as pivotTable"


In [12]:
sql = "Select CertNum, SampleID, SampleType, Reference, DateFinalised, Pass2mm, Pass75um, [WT. SAMPLE] from \
(Select a.CertNum, a.DateFinalised, b.SampleID, b.SampleType, b.Reference, b.Element, b.Value \
from dbo.LabJobCert a join dbo.LabJobResults b on a.CertNum = b.CertNum \
where a.DateFinalised BETWEEN '2017-04-01' and '2017-08-31' and b.Element in ('Pass2mm','Pass75um', 'WT. Sample')) as sourceTable PIVOT \
(MIN(Value) for Element in ([Pass2mm], [Pass75um], [WT. SAMPLE])) as pivotTable"


In [13]:
df = pd.read_sql(sql, engine, index_col = 'DateFinalised')

In [14]:
plt.style.use('seaborn-notebook')
fig = plt.figure(figsize = (8,8))
ax1 = fig.add_subplot(1,1,1)

In [15]:
groups = df.groupby('Reference')
for key, group in groups:
    if key == 'None':
        ax1.plot(group['WT. SAMPLE'], label = key, marker = '+', linestyle = '')
ax1.legend(loc = 'best')    

In [8]:
fig.suptitle('Crushing QC, Percent passed 2mm mesh', fontsize = 16)
ax1.plot(df['Pass2mm'], "b.", label = "_nolegend_")
datarange = ['2017-01-01','2017-07-01']
_5pct = df[df.Pass2mm.notnull()].Pass2mm.astype(np.float).quantile(0.05)
_10pct = df[df.Pass2mm.notnull()].Pass2mm.astype(np.float).quantile(0.1)
ax1.plot(datarange, [_5pct,_5pct], 'b:', label = u'5th percentile', linewidth = 1.5)
ax1.plot(datarange, [_10pct,_10pct], 'b--', label = u'10th percentile', linewidth = 1.5)
ax1.plot((ax1.get_xlim()),(70, 70), 'r-', label = u"Threshold")
ax1.legend(loc = 'best')
ax1.set_xlim(datetime.date(2017,2,27), datetime.date(2017,7,1))
xticks = ax1.get_xticks()
date_labels = [datetime.date.fromordinal(int(f)) for f in xticks]
ax1.set_xticklabels([x.strftime('%d-%b-%y') for x in date_labels], rotation = 45)
ax1.set_ylim((40,100))
ax1.set_ylabel("Passed %")

In [16]:
fig.suptitle(u'Pulverizing QC, Percent passed 75 um mesh', fontsize = 16)
ax1.plot(df['Pass75um'], 'g.',  label = "_nolegend_")
datarange = ['2017-01-01','2017-07-01']
_5pct = df[df.Pass75um.notnull()].Pass75um.astype(np.float).quantile(0.05)
_10pct = df[df.Pass75um.notnull()].Pass75um.astype(np.float).quantile(0.1)
ax1.plot(datarange, [_5pct,_5pct], 'g:', label = u'5th percentile', linewidth = 1.5)
ax1.plot(datarange, [_10pct,_10pct], 'g--', label = u'10th percentile', linewidth = 1.5)
ax1.plot((ax1.get_xlim()), (85, 85), 'r-', label = u'Threshold')
ax1.legend(loc = 'best')  
ax1.set_xlim(datetime.date(2017,2,27), datetime.date(2017,7,1))
xticks = ax1.get_xticks()
date_labels = [datetime.date.fromordinal(int(f)) for f in xticks]
ax1.set_xticklabels([x.strftime('%d-%b-%y') for x in date_labels], rotation = 45)
ax1.set_ylim((50,100))
ax1.set_ylabel("Passed %")

In [99]:
df[df.Pass2mm.notnull()]

,CertNum,SampleID,SampleType,Reference,Pass2mm,Pass75um,WT. SAMPLE,P2mm_val
DateFinalised,,,,,,,,
2017-03-01,TM17028108,Q207710,Original,None,71.8,86.3,50.449,71.8
2017-03-01,TM17028108,Q207748,Original,None,73.8,None,48.411,73.8
2017-03-01,TM17028108,Q207753,Original,None,78.1,None,49.501,78.1
2017-03-01,TM17028108,Q207789,Original,None,72.7,86.3,48.725,72.7
2017-03-01,TM17028108,Q207829,Original,None,78.7,86.4,49.958,78.7
2017-03-01,TM17028108,Q207869,Original,None,65.4,None,49.643,65.4
2017-03-01,TM17028108,Q207871,Original,None,66.6,None,49.521,66.6
2017-03-01,TM17028108,Q207874,Original,None,80.4,None,48.533,80.4
2017-02-28,TM17028123,S143102,Original,None,80.5,86.4,47.656,80.5
